## Manipulating and transforming DataFrames

In [1]:
import warnings

In [26]:
import numpy as np
import pandas as pd
from IPython.display import display
from openbb import obb

In [27]:
print(obb)

OpenBB Platform v4.4.1

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.3.0
    - crypto@1.4.0
    - currency@1.4.0
    - derivatives@1.4.0
    - econometrics@1.4.3
    - economy@1.4.0
    - equity@1.4.0
    - etf@1.4.0
    - fixedincome@1.4.2
    - index@1.4.0
    - news@1.4.0
    - quantitative@1.3.3
    - regulators@1.4.1
    - technical@1.3.3

    - alpha_vantage@1.3.3
    - benzinga@1.4.0
    - biztoc@1.3.3
    - bls@1.1.0
    - cboe@1.3.3
    - cftc@1.1.0
    - ecb@1.3.3
    - econdb@1.3.0
    - federal_reserve@1.4.0
    - finra@1.3.3
    - finviz@1.2.3
    - fmp@1.4.1
    - fred@1.4.1
    - government_us@1.3.3
    - imf@1.1.0
    - intrinio@1.4.0
    - multpl@1.0.3
    - nasdaq@1.3.3
    - oecd@1.4.0
    - polygon@1.4.0
    - sec@1.4

In [3]:
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

Fetches historical price data for the equity "AAPL" using the "yfinance" provider and stores it in 'asset'

In [6]:
asset = obb.equity.price.historical("AAPL", provider="yfinance")

In [7]:
display(asset)

,open,high,low,close,volume,dividend
date,,,,,,
2024-03-25,170.570007,171.940002,169.449997,170.850006,54288300,0.0
2024-03-26,170.000000,171.419998,169.580002,169.710007,57388400,0.0
2024-03-27,170.410004,173.600006,170.110001,173.309998,60273300,0.0
2024-03-28,171.750000,172.229996,170.509995,171.479996,65672700,0.0
2024-04-01,171.190002,171.250000,169.479996,170.029999,46240500,0.0
...,...,...,...,...,...,...
2025-03-18,214.160004,215.149994,211.490005,212.690002,42432400,0.0
2025-03-19,214.220001,218.759995,213.750000,215.240005,54385400,0.0
2025-03-20,213.990005,217.490005,212.220001,214.100006,48862900,0.0


Fetches historical price data for the equity "SPY" using the "yfinance" provider and stores it in 'benchmark'

In [8]:
benchmark = obb.equity.price.historical("SPY", provider="yfinance")

In [9]:
display(benchmark)

,open,high,low,close,volume,dividend
date,,,,,,
2024-03-25,519.799988,520.950012,519.609985,519.770020,48512100,0.000
2024-03-26,521.229980,521.580017,518.400024,518.809998,65463700,0.000
2024-03-27,521.710022,523.210022,519.489990,523.169983,82999800,0.000
2024-03-28,523.210022,524.609985,522.780029,523.070007,96294900,0.000
2024-04-01,523.830017,524.380005,520.969971,522.159973,62477500,0.000
...,...,...,...,...,...,...
2025-03-18,564.799988,565.020020,559.059998,561.020020,66041400,0.000
2025-03-19,562.830017,570.950012,561.630005,567.130005,66556000,0.000
2025-03-20,563.330017,570.570007,562.599976,565.489990,62958200,0.000


Updates the column names for 'asset' and 'benchmark'

In [10]:
columns = [
    "open",
    "high",
    "low",
    "close",
    "volume",
    "dividend"
]

In [11]:
asset.columns = columns
benchmark.columns = columns

Adds a new column 'price_diff' that contains the difference in closing prices

In [12]:
asset["price_diff"] = asset.close.diff()
benchmark["price_diff"] = benchmark.close.diff()

Adds a new column 'gain' that indicates whether the price difference is positive

In [13]:
asset["gain"] = asset.price_diff > 0
benchmark["gain"] = benchmark.price_diff > 0

Adds a new column 'symbol' with the equity symbol

In [14]:
asset["symbol"] = "AAPL"
benchmark["symbol"] = "SPY"

Sets the volume of the 10th row to the mean volume of rows 5 to 9 in 'asset_2'

In [30]:
asset_2 = asset.copy()
asset_2.at[asset_2.index[10], "volume"] = asset_2.volume[asset_2.index[5:10]].mean().astype(int)

In [33]:
asset_2.iat[10, 4]

46041260

Concatenates 'asset' and 'asset_2' and drops duplicates

In [17]:
df = pd.concat([asset, asset_2]).drop_duplicates()

Adds a new column 'returns' that contains the percentage change in closing prices

In [18]:
asset["returns"] = asset["close"].pct_change()

Creates a pivot table from 'asset' with 'returns' as values, 'gain' as columns, and 'sum', 'mean', 'std' as aggregation functions

In [19]:
df = pd.pivot_table(
    data=asset, values="returns", columns="gain", aggfunc=["sum", "mean", "std"]
)

In [20]:
display(df)

sum               mean                std          
gain        False    True      False     True     False     True 
returns -1.262162  1.54811 -0.012021  0.010751  0.01066  0.010631

Concatenates 'asset' and 'benchmark' and groups by 'symbol' to calculate open, high, low, close (OHLC) prices

In [21]:
concated = pd.concat([asset, benchmark])

In [22]:
df = concated.groupby("symbol").close.ohlc()

In [23]:
display(df)

,open,high,low,close
symbol,,,,
AAPL,170.850006,259.019989,165.000000,220.729996
SPY,519.770020,612.929993,495.160004,574.080017


Fetches options chains for the symbol "AAPL" using the "cboe" provider and stores it in 'chains'

In [29]:
chains = obb.derivatives.options.chains("AAPL", provider="cboe")

In [30]:
display(chains)

,underlying_symbol,underlying_price,contract_symbol,expiration,dte,strike,option_type,open_interest,volume,theoretical_price,...,low,prev_close,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho
0,AAPL,220.73,AAPL250328C00100000,2025-03-28,4,100.0,call,41,1,120.5191,...,121.00,115.424999,5.575,0.048300,0.0000,1.0000,0.0000,0.0000,0.0000,0.0164
1,AAPL,220.73,AAPL250328P00100000,2025-03-28,4,100.0,put,869,1,0.0006,...,0.01,0.005000,0.005,1.000000,2.3182,-0.0001,0.0000,-0.0006,0.0000,0.0000
2,AAPL,220.73,AAPL250328C00110000,2025-03-28,4,110.0,call,2,0,110.5270,...,0.00,105.474998,0.000,0.000000,0.0000,0.9999,0.0000,0.0000,0.0001,0.0180
3,AAPL,220.73,AAPL250328P00110000,2025-03-28,4,110.0,put,892,0,0.0009,...,0.00,0.005000,0.000,0.000000,2.1649,-0.0001,0.0000,-0.0009,0.0001,0.0000
4,AAPL,220.73,AAPL250328C00120000,2025-03-28,4,120.0,call,22,0,100.5351,...,0.00,95.474998,0.000,0.000000,0.0000,0.9999,0.0000,0.0000,0.0001,0.0197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,AAPL,220.73,AAPL271217P00430000,2027-12-17,998,430.0,put,0,0,209.5657,...,0.00,214.500000,0.000,0.000000,0.0000,-1.0000,0.0000,-0.0259,0.0000,0.0000
2302,AAPL,220.73,AAPL271217C00435000,2027-12-17,998,435.0,call,0,0,3.2446,...,0.00,3.065000,0.000,0.000000,0.2315,0.0983,0.0021,-0.0067,0.6436,0.4923
2303,AAPL,220.73,AAPL271217P00435000,2027-12-17,998,435.0,put,0,0,214.5657,...,0.00,219.525002,0.000,0.000000,0.0000,-1.0000,0.0000,-0.0259,0.0000,0.0000
2304,AAPL,220.73,AAPL271217C00440000,2027-12-17,998,440.0,call,1769,370,3.0547,...,3.05,2.950000,0.110,0.037288,0.2337,0.0934,0.0020,-0.0065,0.6205,0.4686


Extracts unique expiration dates from 'chains'

In [31]:
expirations = chains.expiration.unique()

Filters 'chains' to get call options expiring on the 6th expiration date and stores it in 'calls'

In [32]:
calls = chains[(chains.option_type == "call") & (chains.expiration == expirations[5])]

Filters 'chains' to get put options expiring on the 6th expiration date and stores it in 'puts'

In [33]:
puts = chains[(chains.option_type == "put") & (chains.expiration == expirations[5])]

Sets the index of 'calls' and 'puts' to 'strike'

In [34]:
calls_strike = calls.set_index("strike")
puts_strike = puts.set_index("strike")

Joins 'calls_strike' and 'puts_strike' on the 'strike' index with a left join, adding suffixes to overlapping column names

In [35]:
joined = calls_strike.join(puts_strike, how="left", lsuffix="_call", rsuffix="_put")

Extracts the last trade prices of call and put options and stores them in 'prices'

In [36]:
prices = joined[["last_trade_price_call", "last_trade_price_put"]]

In [37]:
display(prices)

,last_trade_price_call,last_trade_price_put
strike,,
110.0,0.00,0.04
120.0,0.00,0.12
130.0,0.00,0.08
140.0,0.00,0.11
145.0,69.20,0.14
150.0,70.44,0.25
155.0,0.00,0.24
160.0,55.29,0.26
165.0,0.00,0.31


Adds a new column 'straddle_price' that contains the sum of last trade prices of call and put options

In [38]:
prices.loc[:, "straddle_price"] = prices.sum(axis=1)

In [39]:
display(prices)

,last_trade_price_call,last_trade_price_put,straddle_price
strike,,,
110.0,0.00,0.04,0.04
120.0,0.00,0.12,0.12
130.0,0.00,0.08,0.08
140.0,0.00,0.11,0.11
145.0,69.20,0.14,69.34
150.0,70.44,0.25,70.69
155.0,0.00,0.24,0.24
160.0,55.29,0.26,55.55
165.0,0.00,0.31,0.31


Groups 'chains' by option type, strike, and expiration to calculate the sum of open interest

In [40]:
df = chains.groupby(["option_type", "strike", "expiration"]).open_interest.sum()

In [41]:
display(df)

option_type  strike  expiration
call         5.0     2025-06-20    756
                     2025-09-19     17
                     2025-12-19    208
                     2026-01-16     87
                     2026-06-18     14
                                  ... 
put          440.0   2027-06-17      0
                     2027-12-17      0
             450.0   2026-12-18      0
                     2027-01-15      0
                     2027-06-17      0
Name: open_interest, Length: 2306, dtype: int64

Groups 'chains' by option type, strike, and expiration to calculate the max last trade price and the sum of open interest

In [42]:
df = chains.groupby(["option_type", "strike", "expiration"]).agg(
    {"last_trade_price": "max", "open_interest": "sum"}
)

In [43]:
display(df)

last_trade_price  open_interest
option_type strike expiration                                 
call        5.0    2025-06-20            215.26            756
                   2025-09-19            215.21             17
                   2025-12-19            215.37            208
                   2026-01-16            214.68             87
                   2026-06-18            208.05             14
...                                         ...            ...
put         440.0  2027-06-17            197.65              0
                   2027-12-17              0.00              0
            450.0  2026-12-18            210.76              0
                   2027-01-15            238.00              0
                   2027-06-17            207.60              0

[2306 rows x 2 columns]

Groups 'chains' by option type and calculates the mean bid-ask spread for each group

In [44]:
df = chains.groupby(["option_type"]).apply(
    lambda x: (x["ask"] - x["bid"]).mean()
)

In [45]:
display(df)

option_type
call    1.292220
put     0.945091
dtype: float64

Standardizes the last trade prices by subtracting the mean and dividing by the standard deviation within each expiration group

In [46]:
(
    chains.groupby("expiration").last_trade_price.transform(
        lambda x: (x - x.mean()) / x.std()
    )
)

0       3.647956
1      -0.701411
2       2.999092
3      -0.701411
4       2.704677
          ...   
2301   -0.752972
2302   -0.680155
2303   -0.752972
2304   -0.676137
2305   -0.752972
Name: last_trade_price, Length: 2306, dtype: float64

In [47]:
display(chains)

,underlying_symbol,underlying_price,contract_symbol,expiration,dte,strike,option_type,open_interest,volume,theoretical_price,...,low,prev_close,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho
0,AAPL,220.73,AAPL250328C00100000,2025-03-28,4,100.0,call,41,1,120.5191,...,121.00,115.424999,5.575,0.048300,0.0000,1.0000,0.0000,0.0000,0.0000,0.0164
1,AAPL,220.73,AAPL250328P00100000,2025-03-28,4,100.0,put,869,1,0.0006,...,0.01,0.005000,0.005,1.000000,2.3182,-0.0001,0.0000,-0.0006,0.0000,0.0000
2,AAPL,220.73,AAPL250328C00110000,2025-03-28,4,110.0,call,2,0,110.5270,...,0.00,105.474998,0.000,0.000000,0.0000,0.9999,0.0000,0.0000,0.0001,0.0180
3,AAPL,220.73,AAPL250328P00110000,2025-03-28,4,110.0,put,892,0,0.0009,...,0.00,0.005000,0.000,0.000000,2.1649,-0.0001,0.0000,-0.0009,0.0001,0.0000
4,AAPL,220.73,AAPL250328C00120000,2025-03-28,4,120.0,call,22,0,100.5351,...,0.00,95.474998,0.000,0.000000,0.0000,0.9999,0.0000,0.0000,0.0001,0.0197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,AAPL,220.73,AAPL271217P00430000,2027-12-17,998,430.0,put,0,0,209.5657,...,0.00,214.500000,0.000,0.000000,0.0000,-1.0000,0.0000,-0.0259,0.0000,0.0000
2302,AAPL,220.73,AAPL271217C00435000,2027-12-17,998,435.0,call,0,0,3.2446,...,0.00,3.065000,0.000,0.000000,0.2315,0.0983,0.0021,-0.0067,0.6436,0.4923
2303,AAPL,220.73,AAPL271217P00435000,2027-12-17,998,435.0,put,0,0,214.5657,...,0.00,219.525002,0.000,0.000000,0.0000,-1.0000,0.0000,-0.0259,0.0000,0.0000
2304,AAPL,220.73,AAPL271217C00440000,2027-12-17,998,440.0,call,1769,370,3.0547,...,3.05,2.950000,0.110,0.037288,0.2337,0.0934,0.0020,-0.0065,0.6205,0.4686


**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.